In [1]:
import scrapy
import requests
from bs4 import BeautifulSoup
import datetime
from textblob import TextBlob
import pandas as pd
import json
import numpy as np
import re

In [2]:
def get_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    if soup.find('article') is not None:
        if 'p' in [tag.name for tag in soup.find('article').contents]:
            content_html = soup.find('article')
            content = '\n'.join(tag.text for tag in content_html.contents if tag.name in ['p', 'li', 'h1', 'h2', 'h3', 'ul', 'strong', 'span']).replace('\xa0', '')
        else:
            list_div = list()
            for el in soup.find_all('div'):
                if el.find('p') and not el.find('div'):
                    list_div.append(el)
            index_max = np.argmax([len(block.find_all('p')) for block in list_div])
            content_html = list_div[index_max]
            content = '\n'.join(tag.text for tag in content_html.contents if tag.name in ['p', 'li', 'h1', 'h2', 'h3', 'ul', 'strong', 'span']).replace('\xa0', '')
    else:
        list_div = list()
        for el in soup.find_all('div'):
            if el.find('p') and not el.find('div'):
                list_div.append(el)
        index_max = np.argmax([len(block.find_all('p')) for block in list_div])
        content_html = list_div[index_max]
        content = '\n'.join(tag.text for tag in content_html.contents if tag.name in ['p', 'li', 'h1', 'h2', 'h3', 'ul', 'strong', 'span']).replace('\xa0', '')
    return content_html, content